In [1]:
def find_longest_string(list_of_strings):
    longest_string = None
    longest_string_len = 0     
    for s in list_of_strings:
        if len(s) > longest_string_len:
            longest_string_len = len(s)
            longest_string = s    
    return longest_string

In [2]:
list_of_strings = ['abc', 'python', 'dima']
%time max_length = print(find_longest_string(list_of_strings))

python
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 363 µs


In [3]:
large_list_of_strings = list_of_strings*1000
%time print(find_longest_string(large_list_of_strings))

python
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 350 µs


In [4]:
large_list_of_strings = list_of_strings*100000000
%time max_length = max(large_list_of_strings, key=len)

CPU times: user 21.1 s, sys: 32 ms, total: 21.2 s
Wall time: 21.4 s


In [ ]:
%%time

# step 1:
list_of_string_lens = (len(s) for s in large_list_of_strings)
list_of_string_lens = zip(large_list_of_strings, list_of_string_lens)
#step 2:
max_len = max(list_of_string_lens, key=lambda t: t[1])
print(max_len)

In [ ]:
mapper = len
def reducer(p, c):
    if p[1] > c[1]:
        return p
    return c

In [ ]:
%%time

from functools import reduce
#step 1
mapped = map(mapper, large_list_of_strings)
mapped = zip(large_list_of_strings, mapped)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

In [13]:
## !pip install chunkify #Not working
def chunkify(_list, number_of_chunks=8):
    chunk_size = len(_list) // number_of_chunks
    # return list
    # return [ _list[i*chunk_size:(i+1)*chunk_size] for i in range(number_of_chunks) ]
    # return iterator
    for i in range(number_of_chunks):
        yield _list[i*chunk_size:(i+1)*chunk_size]
    raise StopIteration

In [ ]:
from functools import reduce
data_chunks = chunkify(large_list_of_strings, number_of_chunks=36)
#step 1:
reduced_all = []
for chunk in data_chunks:
    mapped_chunk = map(mapper, chunk)
    mapped_chunk = zip(chunk, mapped_chunk)
    
    reduced_chunk = reduce(reducer, mapped_chunk)
    reduced_all.append(reduced_chunk)
    
#step 2:
reduced = reduce(reducer, reduced_all)
print(reduced)

In [ ]:
%%time

def chunks_mapper(chunk):
    mapped_chunk = map(mapper, chunk) 
    mapped_chunk = zip(chunk, mapped_chunk)
    return reduce(reducer, mapped_chunk)

data_chunks = chunkify(large_list_of_strings, number_of_chunks=30)
#step 1:
mapped = map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

In [ ]:
%%time

from multiprocessing import Pool
pool = Pool(8)
data_chunks = chunkify(large_list_of_strings, number_of_chunks=8)
#step 1:
mapped = pool.map(chunks_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced)

In [6]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups()
data = news.data*10

In [2]:
from collections import Counter
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import re
def clean_word(word):
    return re.sub(r'[^\w\s]','',word).lower()

def word_not_in_stopwords(word):
    return word not in ENGLISH_STOP_WORDS and word and word.isalpha()    
    
def find_top_words(data):
    cnt = Counter()
    for text in data:
        tokens_in_text = text.split()
        tokens_in_text = map(clean_word, tokens_in_text)
        tokens_in_text = filter(word_not_in_stopwords, tokens_in_text)
        cnt.update(tokens_in_text)
        
    return cnt.most_common(10)

In [16]:
%time find_top_words(data)

CPU times: user 1min 30s, sys: 260 ms, total: 1min 31s
Wall time: 1min 30s


[('subject', 122520),
 ('lines', 118240),
 ('organization', 111850),
 ('writes', 78360),
 ('article', 67540),
 ('people', 58320),
 ('dont', 58130),
 ('like', 57570),
 ('just', 55790),
 ('university', 55440)]

In [3]:
def mapper(text):
    tokens_in_text = text.split()
    tokens_in_text = map(clean_word, tokens_in_text)
    tokens_in_text = filter(word_not_in_stopwords, tokens_in_text)
    return Counter(tokens_in_text)
def reducer(cnt1, cnt2):
    cnt1.update(cnt2)
    return cnt1
def chunk_mapper(chunk):
    mapped = map(mapper, chunk)
    reduced = reduce(reducer, mapped)
    return reduced

In [17]:
%%time
from multiprocessing import Pool, cpu_count
from functools import reduce
print("Num of cpus: %d" % cpu_count())
pool = Pool(8)

data_chunks = chunkify(data, number_of_chunks=36)
#step 1:
mapped = pool.map(chunk_mapper, data_chunks)
#step 2:
reduced = reduce(reducer, mapped)
print(reduced.most_common(10))

Num of cpus: 20
[('subject', 122492), ('lines', 118210), ('organization', 111822), ('writes', 78340), ('article', 67524), ('people', 58305), ('dont', 58113), ('like', 57549), ('just', 55779), ('university', 55427)]
CPU times: user 1.61 s, sys: 288 ms, total: 1.9 s
Wall time: 11.6 s


In [ ]:
!lscpu